In [ ]:
import logging
import dspy 
import os
import mlflow
from dotenv import load_dotenv

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.getLogger("LiteLLM").setLevel(logging.WARN)
logging.getLogger("httpx").setLevel(logging.WARN)
logging.getLogger("httpx").setLevel(logging.WARN)
logging.getLogger("urllib3").setLevel(logging.ERROR)
logging.getLogger("dspy").setLevel(logging.INFO)


mlflow.set_tracking_uri("http://mlflow:8080/")
mlflow.set_experiment("deploy_dspy_program")
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

load_dotenv()

True

In [35]:
LLM_URL=os.getenv('LLM_URL')
API_KEY=os.getenv('API_KEY')
LLM_MODEL=os.getenv('LLM_MODEL')
dspy.enable_logging()
lm = dspy.LM(LLM_MODEL,
             api_base=LLM_URL,  # ensure this points to your port
             api_key=API_KEY, model_type='chat')
dspy.configure(lm=lm)
dspy.settings.configure(track_usage=True)

In [36]:
math = dspy.ChainOfThought("question -> answer: float")
x=math(question="Two dice are tossed. What is the probability that the sum equals two?")
print(x)


Prediction(
    reasoning='When two dice are tossed, each die has 6 faces, numbered from 1 to 6. The total number of possible outcomes when two dice are rolled is \\(6 \\times 6 = 36\\).\n\nTo find the probability that the sum of the numbers on the two dice equals 2, we need to determine how many outcomes result in this sum. The only way to achieve a sum of 2 is if both dice show a 1. Therefore, there is only 1 favorable outcome: (1, 1).\n\nThe probability of an event is calculated as the number of favorable outcomes divided by the total number of possible outcomes. Thus, the probability that the sum equals 2 is:\n\n\\[\n\\frac{\\text{Number of favorable outcomes}}{\\text{Total number of possible outcomes}} = \\frac{1}{36}\n\\]\n\nConverting this fraction to a float gives us approximately 0.027777777777777776.',
    answer=0.027777777777777776
)


Trace(request_id=21aa0aeeb9394e80bcc098580cf03823)

In [ ]:
print(x.get_lm_usage())

In [ ]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)
dspy.inspect_history(n=50)

In [ ]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="World War 2")
print(article.get_lm_usage())

In [ ]:
dspy.inspect_history(n=500)

In [ ]:
print(article)

In [ ]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)
dspy.inspect_history(n=80)

In [ ]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)

In [ ]:
dspy.inspect_history(n=80)

In [ ]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context: str = dspy.InputField(desc="facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: bool = dspy.OutputField()
    evidence: dict[str, list[str]] = dspy.OutputField(desc="Supporting evidence for claims")

context = "The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."

text = "Lee scored 3 goals for Colchester United."

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)
dspy.inspect_history(n=80)

In [ ]:
question = "What's something great about the ColBERT retrieval model?"

# 1) Declare with a signature, and pass some config.
classify = dspy.ChainOfThought('question -> answer', n=5)

# 2) Call with input argument.
response = classify(question=question)

# 3) Access the outputs.
response.completions.answer

dspy.inspect_history(n=80)

In [31]:
import dspy
from dspy.datasets import HotPotQA

# logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
# logging.getLogger("dspy").setLevel(logging.DEBUG)

# root = logging.getLogger()
# root.setLevel(logging.DEBUG)
# handler = logging.StreamHandler(sys.stdout)
# handler.setLevel(logging.DEBUG)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
# root.addHandler(handler)

mlflow.set_experiment("optimized_react")

def search(query: str) -> list[str]:
    """Retrieves abstracts from Wikipedia."""
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=50).train]
react = dspy.ReAct("question -> answer", tools=[search])

tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=24)
optimized_react = tp.compile(react, trainset=trainset,requires_permission_to_run=False)

optimized_react.save(path="/home/noelo/dev/dspy-poc/optimized_react.json")

2025/06/02 09:47:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '58b2ce787aa841168c3645a190bc40fe', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/06/02 09:47:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 40

2025/06/02 09:47:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/02 09:47:23 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/02 09:47:23 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


100%|██████████| 10/10 [00:06<00:00,  1.61it/s]


Bootstrapped 4 full traces after 9 examples for up to 1 rounds, amounting to 10 attempts.


Bootstrapping set 4/6


...

2025/06/02 09:51:31 INFO dspy.evaluate.evaluate: Average Metric: 15 / 40 (37.5%)



🏃 View run eval_full_20 at: http://mlflow:8080/#/experiments/34/runs/2c9acc44e7d348a7a7f830e55331ea65
🧪 View experiment at: http://mlflow:8080/#/experiments/34


2025/06/02 09:51:32 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 37.5 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 3', 'Predictor 1: Instruction 1', 'Predictor 1: Few-Shot Set 3'].
2025/06/02 09:51:32 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [20.0, 10.0, 37.5, 10.0, 35.0, 37.5, 25.0, 32.5, 20.0, 35.0, 42.5, 42.5, 42.5, 42.5, 40.0, 5.0, 40.0, 42.5, 40.0, 2.5, 37.5]
2025/06/02 09:51:32 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 42.5
2025/06/02 09:51:32 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/06/02 09:51:32 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 42.5!


🏃 View run funny-donkey-48 at: http://mlflow:8080/#/experiments/34/runs/58b2ce787aa841168c3645a190bc40fe
🧪 View experiment at: http://mlflow:8080/#/experiments/34


[Trace(request_id=440697b656b64ee8a670a6c2bb72a395), Trace(request_id=021d9e9beaca4135b2295b767aecba4a), Trace(request_id=f43feb34d5604c238f1cce7ad2faed2b), Trace(request_id=950b2122b7db4409b6789fdc88905497), Trace(request_id=57038f891d47470992283639f5405c5b), Trace(request_id=65a4ba577f4e47aab575fc3c844cba98), Trace(request_id=1e5f28f9c30f4397a6c8ca74c3a6c66b), Trace(request_id=c98a00969d034fdb8935346977854090), Trace(request_id=dd82d64188b34868bb5044511fc93fef), Trace(request_id=3acaa9618f8c424ca39ffbc32928dfe4)]

In [32]:
print(optimized_react)

react = Predict(StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Given the fields `question`, produce the fields `answer`.\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. You will also have access to your past trajectory, allowing you to build upon previous actions and observations.\n\nYour goal is to use one or more of the supplied tools to gather the necessary information for producing an accurate `answer`.\n\nTo achieve this, you will interleave `next_thought`, `next_tool_name`, and `next_tool_args` in each turn, and also when concluding the task. After each tool call, you will receive an observation, which will be added to your trajectory.\n\nWhen crafting `next_thought`, you may analyze the current situation and devise a plan for future actions. When selecting the `next_tool_name` and its `next_tool_args`, ensure the tool is one of:\n\n1. `search`, which retrieves abstracts from Wikipedia

In [33]:
print(react)

react = Predict(StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Given the fields `question`, produce the fields `answer`.\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search, whose description is <desc>Retrieves abstracts from Wikipedia.</desc>. It takes arguments {'query': {'type': 'string'}} in JSON format.\n(2) fi